In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
iris =load_iris()
X=iris['data']
y=iris['target']
names=iris['target_names']
feature_names=iris['feature_names']
# print(iris)
# print(names)
# print(feature_names)
scaler =StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split (
    X_scaled, y, test_size=0.2, random_state=2
)

In [17]:
#====================================================
#====================================================
#==============  CONfIG NEURAL NETWORK ==============
#====================================================
import torch
import torch.nn.functional as F
import torch.nn as nn
#from torch.autograd import Variable
class Model (nn.Module): #Model that inherits from nn.Module
    def __init__(self, input_dim):
        """
        def __init__(self, input_dim):: This defines the constructor 
        (__init__) method of the Model class. It takes an argument input_dim, 
        which specifies the dimensionality of the input data.
        """
        super(Model, self).__init__()
        """
        super(Model, self).__init__(): 
        This line calls the constructor of the parent class (nn.Module) 
        to initialize the base functionalities of the neural network.
        """
        self.layer1 = nn.Linear(input_dim,50)
        self.layer2 = nn.Linear(50,50)
        self.layer3 = nn.Linear(50,3)
        
    def forward(self,x):
        """
        def forward(self, x):
            This line defines the forward method of the Model class. 
            This method defines the forward pass of the neural network,
            which calculates the output for a given input.
        """
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        """
        x represents the class probabilities for the input data.
        """
        return x
print(X_train.shape)
print(X_test.shape)

model = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
print(model)
        

(120, 4)
(30, 4)
Model(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)


In [18]:
#====================================================
#====================================================
#==============  Epochs                ==============
#====================================================

import tqdm
EPOCHS = 100

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test  = torch.from_numpy(X_test).float()
y_test  = torch.from_numpy(y_test).long()
y_test

tensor([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1,
        1, 0, 0, 2, 0, 2])

In [21]:
loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

# print(loss_list)
for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred,y_train)
    loss_list[epoch] = loss.item()
    
    optimizer.zero_grad() 
    """
    Sets the gradients of all model parameters to zero before backpropagation.
    This is necessary to avoid accumulating gradients from previous iterations.
    
    
    """
    loss.backward()
    """
    performs backpropagation through the computational graph,
    calculating the gradients of the loss function with respect to each model parameter.
    
    the gradients (partial derivatives) in Deep Learning:
    Gradients are multidimensional arrays that represent the rate of change of a function (loss function in this case) 
    with respect to each element of its input.
    In deep learning models, the loss function measures how well the model's predictions align with the actual labels.
    We want to minimize this loss function.
    Gradients tell us how much each parameter (weight or bias) in the model contributes to the overall loss.
    By understanding these contributions, we can update the parameters in a direction that minimizes the loss,
    essentially improving the model's performance
    """
    optimizer.step()
    """
    Updates the model parameters based on the calculated gradients using the chosen optimizer (e.g., Adam, SGD).
    This essentially adjusts the weights and biases in the model based on the errors observed in the predictions.

    The optimizer uses the calculated gradients to update the model parameters in a direction that minimizes the loss function.
    """
    
    with torch.no_grad():
        """
        This context manager disables gradient calculation during the evaluation step.
        Since we're not updating the model parameters here,
        calculating gradients is unnecessary and can waste computation.
        """
        y_pred = model(X_test)
        # print(y_pred)
        # print(y_pred.shape)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        print(correct)
        accuracy_list[epoch] = correct.mean()
        print(accuracy_list[epoch])
        

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 614.62it/s]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
0.9333333373069763
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
0.9333333373069763
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
0.9333333373069763
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
0.9333333373069763
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
0.9333333373069763
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
0.9333333373069763
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [20]:
#------------------------------------------------------------------
#------------------------    PREDICT ------------------------------
#------------------------------------------------------------------
#------------------------------------------------------------------
# Input a sample data
sample_data = np.array([[5.1, 3.5, 1.4, 0.2]])

# Scale the sample data using the same scaler used for training
sample_data_scaled = scaler.transform(sample_data)

# Make predictions using the trained model
with torch.no_grad():
    output = model(torch.from_numpy(sample_data_scaled).float())
    print(output)
    predicted = torch.argmax(output, dim=1)
    print(predicted)
    print(predicted.item())

# Print the predicted iris label

# Print the predicted iris labels
predicted_labels = np.array(names)[predicted.item()]
print(predicted_labels)

tensor([[9.9380e-01, 5.8271e-03, 3.7732e-04]])
tensor([0])
0
setosa
